# ViT: An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale

- **ViT(Vision Transformer)**: Google에서 개발하였으며, CNN의 의존 없이 Transformer를 CV(Computer Vision)에 적용하여 이미지 분류에서 높은 성능을 낸 모델

# 0. Abstract

- ViT의 배경
    - **Transformer**는 사실상 NLP의 표준이 되었다.
    - 그러나 CV(Computer Vision)에 적용하는 데에는 제한이 있었다.
    - CV에서 Transformer는 CNN과 같이 쓰이는 경향이 있었다.
- ViT의 성능
    - 논문은 **CNN의 의존 없이** 순수한 **Transformer를 적용**하였다고 밝힌다.
    - 많은 양의 데이터로 사전 훈련한 ViT 모델은 ImageNet, CIFAR-100 등의 작은 크기의 이미지 분류에서 SOTA를 달성하였다.|

# 1. Introduction

- Transformer의 장점
    - Transformer의 계산 효율성과 확장성에 의해 다음이 가능해졌다.
        1. 100B parameters가 넘는 매우 큰 크기의 모델도 학습이 가능해졌다.
        2. 성능 포화의 징후가 없는 것처럼 보인다.

- CV에서의 Transformer 적용 노력
    - NLP에서의 Transformer의 성공을 보고 CV에서도 Transformer를 다음과 같이 적용하고자 했다.
        1. CNN과 self-attention의 결합
        2. 특수한 패턴을 갖는 attention으로 CNN을 완전히 대체
    - 그러나 CV로의 효과적인 확장을 보여주지는 못했다.

- **ViT에서의 Transformer 적용 방식**
    - ViT에서는 다음과 같이 Transformer를 적용하고자 했다.
        1. **이미지를 patch로 분할**
        2. **patch를 linear embedding**하여 Transformer에 입력으로 제공
    - 이렇게 함으로써 **Image patch를 마치 NLP의 token처럼** 다룰 수 있게 되었다.

- **ViT의 성과**
    - 중간 규모의 ImageNet으로 훈련했을 때에는 ViT가 ResNet보다 정확도가 다소 낮았다.
    - 그러나 **대규모 데이터셋**(예: ImageNet-21k, 14M-300M images)에서 **사전훈련**된 ViT는 ImageNet에서 ResNet의 성능을 뛰어넘었다!
    - ImageNet에서 88.55%의 정확도를 달성했다.

# 2. Method

- ViT 모델은 Transformer의 구조와 최대한 가깝게 따라하고자 했다.
    - 그 이유는 NLP에서 Transformer의 장점을 그대로 CV에 사용하기 위해서이다.

## 2.1. Vision Transformer(ViT)

### 2.1.1. ViT Architecture

- ViT의 모델 구조는 다음과 같다.

<img src="01.png" width=70% height=70%>

- 기본적으로 ViT는 **pre-training**과 **fine-tuning** 과정으로 나뉜다.
    - **pre-training**: 대규모 데이터로 사전 학습한다.
    - **fine-tuning**: 중소 규모의 데이터로 학습한다.
    - 두 과정에서의 모델 구조는 **MLP Head(Classification Head)를 제외하고 동일**하다.

- ViT는 다음과 같은 과정으로 학습이 진행된다.

- 1. **이미지를 patch로 자른 뒤 배열한다.**
    - Transformer는 1D sequence를 입력으로 받으므로 **2D image를 1D로 처리**해야 한다.
    - $\mathbb{R}^{H\times W\times C} \rightarrow \mathbb{R}^{N\times (P^2 \times C)}$ ($(H, W)$: image 해상도, $C$: channel, $N$: patch 개수, $(P, P)$: patch 해상도)


- 2. **trainable linear projection으로 patch를 $D$ dimension으로 매핑한다.**
    - Transformer의 모든 latent vector의 크기인 $D$ 차원에 맞게 linear projection을 해준다.
    - 이때, linear projection 이후 **patch embedding의 맨 앞**에 **BERT의 [CLS] token 역할을 하는 embedding**(이하 **class embedding**)를 붙인다.
        - class embedding에 대응되는 Transformer의 output은 BERT처럼 실제로 classification에 이용한다!

- 3. **Position embedding을 patch embedding에 더한다.**
    - 실험 결과 2D embedding(예: (1, 1), (1, 2), ...)보다 **1D embedding**(예: 1, 2, 3, ...)의 성능이 더 좋았다고 한다.

- 4. **embedding sequence를 Transformer에 입력한다.**
    - Transformer 구조는 위 그림에서 볼 수 있다.
        - Transformer와는 Normalization 순서가 다르다.
        - **Layer Norm** 을 사용했다.
        - MLP는 2개의 layer로 구성되며, **GELU 활성화 함수**를 적용했다.

- 5. **class embedding에 대응되는 Transformer의 output을 classification head로 이동한다.**
    - **classification head**는 pre-training과 fine-tuning에서 차이가 있다.
        1. pre-training: MLP with 1 hidden layer
        2. fine-tuning: linear layer
    - classification head를 지나면 최종적으로 classification을 수행하게 된다.

- 위 과정을 식으로 정리하면 다음과 같다.

<img src="02.png">

- **MSA**: Multi-head Self Attention
- **MLP**: Multi-layer Perceptron
- **LN**: Layer Norm

### 2.1.2. Inductive bias

- **Inductive bias**: 새로운 데이터에 대한 일반적인 추론을 하기 위한 모델의 가정
    - 예: CNN은 모델 특성상 2차원의 이웃한 데이터를 분석하기 때문에 지역성이라는 inductive bias를 가진다고 볼 수 있다.
    - 예: MLP는 2차원 이미지를 1차원으로 입력하므로 지역성이라는 이미지의 특성을 분석하기 어렵다. 따라서 이미지에서는 CNN보다 inductive bias가 적다.

- **Vision Transformer**의 self-attention이나 MLP는 2차원 이미지의 특성을 적극적으로 활용하지 못하고 있으며, position embedding 또한 1D embedding을 사용하므로 **inductive bias가 적다**고 볼 수 있다.
    - ViT에서 inductive bias가 존재하는 부분은 다음과 같다.
        1. patch를 잘라서 순서대로 넣는 부분
        2. fine-tuning에서의 position embedding 조절

### 2.1.3. Hybrid Architecture

- **Hybrid Architecture**: 기존의 image patch 대신, image의 CNN feature map으로부터 patch를 얻을 수도 있다.
    - CNN으로 인해 각 feature의 원소들은 2차원 정보를 포함하고 있다. 따라서 극단적으로 1x1 patch를 이용해도 된다.


## 2.2. Fine-tuning and Higher Resolution

- **Fine-tuning에서의 모델 구조**
    - pre-training과 다른 점은 딱 하나, **classification head**가 MLP에서 linear layer로 바뀐다는 점이다.
    - linear layer는 가중치가 0으로 초기화된 $D \times K$ feedfforward layer이다. ($K$: number of classes)

- **Higher Resolution**
    - ViT 모델은 pre-training 데이터보다 **더 높은 해상도를 가진 데이터**를 다룰 수 있다.
    - 그런데 이 경우 더이상 pre-training에서의 position embedding이 의미가 없어진다.
    - 따라서 fine-tuning 때에는 **2D interpolation**으로 position embedding을 수행하여 2D 위치 정보를 담는다!
        - 이 부분에서 inductive bias가 있다고 할 수 있다.

# 3. Experiments

- ViT-Base, ViT-Large, ViT-Huge는 다음과 같은 모델 변수를 갖는다.

<img src="03.png" width=70% height=70%>

- 기존의 BiT 모델과 비교한 성능 지표는 다음과 같다.

<img src="04.png" width=70% height=70%>

- 결과적으로 순수한 Transformer만을 이용하여 CV task에서 SOTA를 달성하였다.

**[참고 자료]**
1. An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale 논문: https://arxiv.org/abs/2010.11929
2. [Paper Review] ViT: An Image is Worth 16x16 Words:Transformers for Image Recognition at Scale, Youtube: https://www.youtube.com/watch?v=0kgDve_vC1o